# Trino Data Vault Test Notebook
Copyright (C) 2021 OS-Climate

This sample demonstrates:
* Configuration of access for both column-level and row-level data in Trino
* Testing / validate access controls definition, using 3 separate user profiles with various levels of access control


Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools

In [2]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import pandas as pd
import uuid
import trino
import osc_ingest_trino as osc
from sqlalchemy.engine import create_engine

Load environment variables

In [3]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [4]:
print(os.environ['TRINO_USER_USER1'])
print(os.environ['TRINO_USER_USER2'])
print(os.environ['TRINO_USER_USER3'])

os-climate-user1
os-climate-user2
os-climate-user3


Create a simple data frame with list of power plans and capacity for ingestion

In [5]:
# initialise data of lists of powerplants and capacity
data = {'country':['France', 'France', 'France', 'France', 'Australia', 'Australia'],
        'name':['Albas', 'Albi', 'Albias', 'Allaire', 'Tower Mine', 'Townsville'],
        'capacity_gwh':[1.8, 6.70448, 2.41, 8.2, 41.2, 220.0],
        'rating':['A', 'B', 'A', 'C', 'B', 'A']}
# Create DataFrame
df = pd.DataFrame(data)
# Add a unique identifier to the data set
uid = str(uuid.uuid4())
df['uuid'] = uid
# Print the output
df = df.convert_dtypes()
df

,country,name,capacity_gwh,rating,uuid
0,France,Albas,1.8,A,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
1,France,Albi,6.70448,B,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
2,France,Albias,2.41,A,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
3,France,Allaire,8.2,C,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
4,Australia,Tower Mine,41.2,B,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
5,Australia,Townsville,220.0,A,d7e632cc-4bf5-42e0-84dd-b5ab4dece473


In [6]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       6 non-null      string 
 1   name          6 non-null      string 
 2   capacity_gwh  6 non-null      Float64
 3   rating        6 non-null      string 
 4   uuid          6 non-null      string 
dtypes: Float64(1), string(4)
memory usage: 374.0 bytes


Define variables for catalog / schema / table names for ingestion

In [7]:
catalogname = 'osc_datacommons_dev'
schemaname = 'demo'
tablename = 'gppd'

Connect to S3 Dev Bucket and upload the data as parquet for ingestion 

In [8]:
import boto3
s3 = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ["S3_DEV_ENDPOINT"],
    aws_access_key_id=os.environ["S3_DEV_ACCESS_KEY"],
    aws_secret_access_key=os.environ["S3_DEV_SECRET_KEY"],
)
bucket = s3.Bucket(os.environ["S3_DEV_BUCKET"])

In [9]:
tmp = f'/tmp/{tablename}.parquet'
df.to_parquet(tmp,index=False)
bucket.upload_file(tmp, f'trino/{schemaname}/{tablename}/{tablename}.parquet')

Open Trino connection to load the data with os-climate-user1

In [10]:
sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER_USER1'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD_USER1']),
    'http_scheme': 'https'
}
engine_dev = create_engine(sqlstring, connect_args = sqlargs)
print("connecting with engine " + str(engine_dev))
connection_dev = engine_dev.connect()

connecting with engine Engine(trino://os-climate-user1@trino-secure-odh-trino.apps.odh-cl1.apps.os-climate.org:443/)


In [11]:
schema_create_sql = f"""
create schema if not exists {catalogname}.{schemaname}
"""
schema_create = engine_dev.execute(schema_create_sql)
print(schema_create.fetchall())

[(True,)]


In [12]:
table_schema = osc.create_table_schema_pairs(df)

sql = f"""
create table if not exists {catalogname}.{schemaname}.{tablename}(
{table_schema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket.name}/trino/{schemaname}/{tablename}/'
)
"""
print(sql)
qres = engine_dev.execute(sql)
print(qres.fetchall())


create table if not exists osc_datacommons_dev.demo.gppd(
    country varchar,
    name varchar,
    capacity_gwh double,
    rating varchar,
    uuid varchar
) with (
    format = 'parquet',
    external_location = 's3a://ocp-odh-os-demo-s3/trino/demo/gppd/'
)

[(True,)]


Create and ingest dataset metadata

In [13]:
# Initialize metadata
metadata = {'schema_name':[schemaname],
            'dataset_name':[tablename],
            'dataset_id':[uid],
            'dataset_title':['Global Power Plant Database'],
            'dataset_version': ['1.3.0'] }
# Create DataFrame
df_meta = pd.DataFrame(metadata)
# Normliaze and print the data frame
df_meta = df_meta.convert_dtypes()
df_meta

,schema_name,dataset_name,dataset_id,dataset_title,dataset_version
0,demo,gppd,d7e632cc-4bf5-42e0-84dd-b5ab4dece473,Global Power Plant Database,1.3.0


In [14]:
meta_schema_name = 'demo'
meta_table_name_dataset = 'demo_metastore'

In [15]:
tmp = f'/tmp/{meta_table_name_dataset}.parquet'
df_meta.to_parquet(tmp,index=False)
bucket.upload_file(tmp, f'trino/{meta_schema_name}/{meta_table_name_dataset}/{meta_table_name_dataset}.parquet')

In [16]:
table_schema_meta = osc.create_table_schema_pairs(df_meta)

sql_meta = f"""
create table if not exists {catalogname}.{meta_schema_name}.{meta_table_name_dataset}(
{table_schema_meta}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket.name}/trino/{meta_schema_name}/{meta_table_name_dataset}/'
)
"""
print(sql_meta)
qres_meta = engine_dev.execute(sql_meta)
print(qres_meta.fetchall())


create table if not exists osc_datacommons_dev.demo.demo_metastore(
    schema_name varchar,
    dataset_name varchar,
    dataset_id varchar,
    dataset_title varchar,
    dataset_version varchar
) with (
    format = 'parquet',
    external_location = 's3a://ocp-odh-os-demo-s3/trino/demo/demo_metastore/'
)

[(True,)]


Query the data with os-climate-user1

In [17]:
sql = f"""
select * from {catalogname}.{schemaname}.{tablename}
"""
df_read = pd.read_sql(sql, engine_dev)
df_read

,country,name,capacity_gwh,rating,uuid
0,France,Albas,1.80000,A,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
1,France,Albi,6.70448,B,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
2,France,Albias,2.41000,A,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
3,France,Allaire,8.20000,C,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
4,Australia,Tower Mine,41.20000,B,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
5,Australia,Townsville,220.00000,A,d7e632cc-4bf5-42e0-84dd-b5ab4dece473


File system access control in Trino is driven with authorization rules managed in trino-config-secret.yml. For more information on rules configuration:

https://trino.io/docs/current/security/file-system-access-control.html

In this demo, we use table rules to restrict access to the gppd table created in the demo schema (open to anyone). In particular:

os-climate-user1 belongs to the group demo_dv_dev and has full privileges on gppd table:

```
{
        "group": "demo_dv_dev",
        "catalog": "osc_datacommons_dev",
        "schema": "demo",
        "table": "gppd",
        "privileges": ["SELECT", "INSERT", "DELETE", "OWNERSHIP"]
}
```

os-climate-user2 belongs to the group demo_dv_quant and has SELECT privileges only for the power plants located in France:

```
{
        "group": "demo_dv_quant",
        "catalog": "osc_datacommons_dev",
        "schema": "demo",
        "table": "gppd",
        "privileges": ["SELECT"],
        "filter": "country = \"France\""
}
```

os-climate-user3 belongs to the group demo_dv_user', has SELECT privileges only for the power plants located in France with restriction on the field rating (which is proprietary data):

```
{
        "group": "demo_dv_user",
        "catalog": "osc_datacommons_dev",
        "schema": "demo",
        "table": "gppd",
        "privileges": ["SELECT"],
        "filter": "country = \'France\'",
        "columns": [
          {
            "name": "rating",
            "allow": false
          }
        ]
}
```

Query the data with os-climate-user2 - returns only rows for power plants located in France

In [18]:
sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER_USER2'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD_USER2']),
    'http_scheme': 'https'
}
engine_scientist = create_engine(sqlstring, connect_args = sqlargs)
print("connecting with engine " + str(engine_scientist))
connection_scientist = engine_scientist.connect()

connecting with engine Engine(trino://os-climate-user2@trino-secure-odh-trino.apps.odh-cl1.apps.os-climate.org:443/)


In [19]:
df_read2 = pd.read_sql(sql, engine_scientist)
df_read2

,country,name,capacity_gwh,rating,uuid
0,France,Albas,1.80000,A,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
1,France,Albi,6.70448,B,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
2,France,Albias,2.41000,A,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
3,France,Allaire,8.20000,C,d7e632cc-4bf5-42e0-84dd-b5ab4dece473


Query the data with os-climate-user2 - throws an error if the rating field is queried, otherwise returns the data set for plants located in France

In [20]:
sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER_USER3'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD_USER3']),
    'http_scheme': 'https'
}
engine_user = create_engine(sqlstring, connect_args = sqlargs)
print("connecting with engine " + str(engine_scientist))
connection_user = engine_user.connect()

connecting with engine Engine(trino://os-climate-user2@trino-secure-odh-trino.apps.odh-cl1.apps.os-climate.org:443/)


In [21]:
df_read3 = pd.read_sql(sql, engine_user)
df_read3

TrinoUserError: TrinoUserError(type=USER_ERROR, name=PERMISSION_DENIED, message="Access Denied: Cannot select from table osc_datacommons_dev.demo.gppd", query_id=20211210_083514_00140_givsz)

In [22]:
sql = f"""
select country, name, capacity_gwh, uuid from {catalogname}.{schemaname}.{tablename}
"""
df_read3 = pd.read_sql(sql, engine_user)
df_read3

,country,name,capacity_gwh,uuid
0,France,Albas,1.80000,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
1,France,Albi,6.70448,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
2,France,Albias,2.41000,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
3,France,Allaire,8.20000,d7e632cc-4bf5-42e0-84dd-b5ab4dece473
